In [39]:
# Load the extension that allows us to compile CUDA code in python notebooks
# Documentation is here: https://nvcc4jupyter.readthedocs.io/en/latest/
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter



  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-9cgjep6z
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-9cgjep6z
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


# 新段落

In [40]:
import torchvision
import os

def download_mnist_dataset():
    # 创建目录
    os.makedirs("train_mnist/MNIST/raw", exist_ok=True)
    os.makedirs("test_mnist/MNIST/raw", exist_ok=True)

    # 下载训练数据
    train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True)
    test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True)

    print("MNIST dataset downloaded successfully.")

# 调用函数下载数据集
download_mnist_dataset()

MNIST dataset downloaded successfully.


In [41]:
'''DO NOT UNCOMMENT THIS CELL unless you are running this notebook on Google Colab'''
# from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)


'DO NOT UNCOMMENT THIS CELL unless you are running this notebook on Google Colab'

In [42]:
%%cuda_group_save -g "knn" -n "main.cu"

// Required header files
#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <cstring>
#include <algorithm>
#include <cuda_runtime.h>
#include <cfloat>
#include <chrono>

// Constants definition
#define THREADS 256
#define IMAGESIZE 784

// Error checking macro
#define CUDA_CHECK(call) { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        printf("CUDA error %d: %s at %s:%d\n", err, cudaGetErrorString(err), __FILE__, __LINE__); \
        exit(1); \
    } \
}

struct TrainingSample {
    int label;
    float image[IMAGESIZE];
};

uint32_t swap32(uint32_t val) {
    val = ((val << 8) & 0xFF00FF00) | ((val >> 8) & 0xFF00FF);
    return (val << 16) | (val >> 16);
}

__global__ void bitonicSortStep(float* d_distances, int* d_labels, int j, int k, int num_samples) {
    unsigned int i = threadIdx.x + blockDim.x * blockIdx.x;
    if (i >= num_samples) return;

    unsigned int ixj = i ^ j;
    if (ixj > i && ixj < num_samples) {
        if ((i & k) == 0) {
            if (d_distances[i] > d_distances[ixj]) {
                float temp_dist = d_distances[i];
                d_distances[i] = d_distances[ixj];
                d_distances[ixj] = temp_dist;

                int temp_label = d_labels[i];
                d_labels[i] = d_labels[ixj];
                d_labels[ixj] = temp_label;
            }
        } else {
            if (d_distances[i] < d_distances[ixj]) {
                float temp_dist = d_distances[i];
                d_distances[i] = d_distances[ixj];
                d_distances[ixj] = temp_dist;

                int temp_label = d_labels[i];
                d_labels[i] = d_labels[ixj];
                d_labels[ixj] = temp_label;
            }
        }
    }
}

void bitonicSort(float* d_distances, int* d_labels, int num_samples, cudaStream_t stream) {
    int pow2_size = 1;
    while (pow2_size < num_samples) pow2_size <<= 1;

    dim3 block(256);  // Fixed thread block size
    dim3 grid((pow2_size + block.x - 1) / block.x);  // Calculate grid size

    // Main sorting loops
    for (int k = 2; k <= pow2_size; k <<= 1) {
        for (int j = k >> 1; j > 0; j >>= 1) {
            bitonicSortStep<<<grid, block, 0, stream>>>(
                d_distances, d_labels, j, k, num_samples
            );
            CUDA_CHECK(cudaGetLastError());
        }
    }
}

__global__ void computeDistances(float* d_images, float* d_testImage,
                               float* d_distances, int* d_labels,
                               int* d_train_labels, int num_samples) {
    int tid = threadIdx.x;
    int bid = blockIdx.x;
    int idx = bid * blockDim.x + tid;
    
    if (idx < num_samples) {
        float sum = 0.0f;
        
        // 使用向量化加载优化计算
        float4* train_vec = (float4*)(&d_images[idx * IMAGESIZE]);
        float4* test_vec = (float4*)d_testImage;
        
        // IMAGESIZE = 784 = 196 * 4 向量化处理前784个元素
        #pragma unroll 8
        for (int i = 0; i < IMAGESIZE/4; i++) {
            float4 train = train_vec[i];
            float4 test = test_vec[i];
            
            float diff_x = train.x - test.x;
            float diff_y = train.y - test.y;
            float diff_z = train.z - test.z;
            float diff_w = train.w - test.w;
            
            sum += diff_x * diff_x + diff_y * diff_y + 
                   diff_z * diff_z + diff_w * diff_w;
        }
        
        // 处理剩余的元素
        for (int i = (IMAGESIZE/4)*4; i < IMAGESIZE; i++) {
            float diff = d_images[idx * IMAGESIZE + i] - d_testImage[i];
            sum += diff * diff;
        }
        
        d_distances[idx] = sqrtf(sum);
        d_labels[idx] = d_train_labels[idx];
    }
}

bool loadMNISTImages(const std::string& image_path, const std::string& label_path,
                    std::vector<TrainingSample>& samples) {
    std::ifstream image_file(image_path, std::ios::binary);
    if (!image_file) {
        std::cerr << "Cannot open image file: " << image_path << std::endl;
        return false;
    }

    std::ifstream label_file(label_path, std::ios::binary);
    if (!label_file) {
        std::cerr << "Cannot open label file: " << label_path << std::endl;
        return false;
    }

    uint32_t magic, num_items, num_rows, num_cols;
    image_file.read(reinterpret_cast<char*>(&magic), sizeof(magic));
    image_file.read(reinterpret_cast<char*>(&num_items), sizeof(num_items));
    image_file.read(reinterpret_cast<char*>(&num_rows), sizeof(num_rows));
    image_file.read(reinterpret_cast<char*>(&num_cols), sizeof(num_cols));

    magic = swap32(magic);
    num_items = swap32(num_items);
    num_rows = swap32(num_rows);
    num_cols = swap32(num_cols);

    uint32_t label_magic, num_labels;
    label_file.read(reinterpret_cast<char*>(&label_magic), sizeof(label_magic));
    label_file.read(reinterpret_cast<char*>(&num_labels), sizeof(num_labels));

    label_magic = swap32(label_magic);
    num_labels = swap32(num_labels);

    samples.resize(num_items);
    std::vector<unsigned char> pixels(num_rows * num_cols);

    for (uint32_t i = 0; i < num_items; ++i) {
        unsigned char label;
        label_file.read(reinterpret_cast<char*>(&label), 1);
        samples[i].label = static_cast<int>(label);

        image_file.read(reinterpret_cast<char*>(pixels.data()), pixels.size());

        for (size_t j = 0; j < pixels.size(); ++j) {
            samples[i].image[j] = static_cast<float>(pixels[j]) / 255.0f;
        }

        if (i % 1000 == 0) {
            std::cout << "\rLoading data: " << (i * 100.0f / num_items) << "%" << std::flush;
        }
    }
    std::cout << "\rLoading data: 100%" << std::endl;

    return true;
}

int main() {
    // Start timing for performance measurement
    // 开始计时，用于性能测量
    auto start_time = std::chrono::high_resolution_clock::now();
    
    // Vectors to store training and test data
    // 用于存储训练和测试数据的向量
    std::vector<TrainingSample> train_samples;
    std::vector<TrainingSample> test_samples;

    // Load training data from MNIST dataset
    // 从MNIST数据集加载训练数据
    if (!loadMNISTImages("./data/MNIST/raw/train-images-idx3-ubyte",
                        "./data/MNIST/raw/train-labels-idx1-ubyte",
                        train_samples)) {
        return -1;
    }
    std::cout << "Successfully loaded " << train_samples.size() << " training samples." << std::endl;

    // Load test data from MNIST dataset
    // 从MNIST数据集加载测试数据
    if (!loadMNISTImages("./data/MNIST/raw/t10k-images-idx3-ubyte",
                        "./data/MNIST/raw/t10k-labels-idx1-ubyte",
                        test_samples)) {
        return -1;
    }
    std::cout << "Successfully loaded " << test_samples.size() << " testing samples." << std::endl;

    // Get the number of samples
    // 获取样本数量
    int num_trainsamples = train_samples.size();
    int num_testsamples = test_samples.size();

    // Allocate page-locked (pinned) memory for better transfer speed
    // 分配页锁定内存（固定内存）以获得更好的传输速度
    float* h_train_images;
    int* h_train_labels;
    CUDA_CHECK(cudaMallocHost(&h_train_images, num_trainsamples * IMAGESIZE * sizeof(float)));
    CUDA_CHECK(cudaMallocHost(&h_train_labels, num_trainsamples * sizeof(int)));

    // Copy data to page-locked memory
    // 将数据复制到页锁定内存
    for (int i = 0; i < num_trainsamples; ++i) {
        h_train_labels[i] = train_samples[i].label;
        std::memcpy(&h_train_images[i * IMAGESIZE], train_samples[i].image, IMAGESIZE * sizeof(float));
    }

    // Allocate GPU memory with proper alignment for vectorized access
    // 分配GPU内存，确保向量化访问的正确对齐
    float* d_train_images;
    int* d_train_labels;
    size_t pitch;  // Pitch for aligned memory allocation / 对齐内存分配的间距
    CUDA_CHECK(cudaMallocPitch((void**)&d_train_images, &pitch,
                              IMAGESIZE * sizeof(float), num_trainsamples));
    CUDA_CHECK(cudaMalloc(&d_train_labels, num_trainsamples * sizeof(int)));

    // Copy training data to GPU
    // 将训练数据复制到GPU
    CUDA_CHECK(cudaMemcpy(d_train_images, h_train_images, 
                         num_trainsamples * IMAGESIZE * sizeof(float), 
                         cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_train_labels, h_train_labels, 
                         num_trainsamples * sizeof(int), 
                         cudaMemcpyHostToDevice));

    // KNN algorithm parameters
    // KNN算法参数
    const int k = 5;  // Number of nearest neighbors / 最近邻的数量
    int correct_predictions = 0;  // Counter for correct predictions / 正确预测的计数器
    
    // Allocate GPU memory for test processing
    // 为测试处理分配GPU内存
    float* d_test_image;  // Current test image / 当前测试图像
    float* d_distances;   // Distances to all training samples / 到所有训练样本的距离
    int* d_sort_labels;   // Labels for sorting / 用于排序的标签
    float* h_distances;   // Host memory for distances / 主机端的距离内存
    int* h_labels;        // Host memory for labels / 主机端的标签内存
    
    // Ensure memory alignment for vectorized access
    // 确保内存对齐以进行向量化访问
    CUDA_CHECK(cudaMalloc(&d_test_image, ((IMAGESIZE + 3) / 4) * 4 * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_distances, num_trainsamples * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_sort_labels, num_trainsamples * sizeof(int)));
    CUDA_CHECK(cudaMallocHost(&h_distances, k * sizeof(float)));
    CUDA_CHECK(cudaMallocHost(&h_labels, k * sizeof(int)));

    // Configure kernel execution parameters
    // 配置核函数执行参数
    const int threadsPerBlock = 256;  // Threads per block / 每个块的线程数
    int blocksPerGrid = (num_trainsamples + threadsPerBlock - 1) / threadsPerBlock;  // Number of blocks / 块的数量

    // Process each test sample
    // 处理每个测试样本
    for (int t = 0; t < num_testsamples; t++) {
        // Copy current test image to GPU
        // 将当前测试图像复制到GPU
        CUDA_CHECK(cudaMemcpy(d_test_image, test_samples[t].image,
                            IMAGESIZE * sizeof(float), cudaMemcpyHostToDevice));

        // Compute distances between test image and all training images
        // 计算测试图像与所有训练图像之间的距离
        computeDistances<<<blocksPerGrid, threadsPerBlock>>>(
            d_train_images,
            d_test_image,
            d_distances,
            d_sort_labels,
            d_train_labels,
            num_trainsamples
        );
        CUDA_CHECK(cudaGetLastError());

        // Sort distances to find k nearest neighbors
        // 对距离进行排序以找到k个最近邻
        bitonicSort(d_distances, d_sort_labels, num_trainsamples, 0);

        // Copy k nearest neighbors back to host
        // 将k个最近邻的结果复制回主机
        CUDA_CHECK(cudaMemcpy(h_distances, d_distances, k * sizeof(float), cudaMemcpyDeviceToHost));
        CUDA_CHECK(cudaMemcpy(h_labels, d_sort_labels, k * sizeof(int), cudaMemcpyDeviceToHost));

        // Majority voting among k nearest neighbors
        // k个最近邻中的多数投票
        std::vector<int> labelCount(10, 0);  // Count votes for each digit / 统计每个数字的票数
        for (int i = 0; i < k; ++i) {
            if (h_labels[i] >= 0 && h_labels[i] < 10) {
                labelCount[h_labels[i]]++;
            }
        }

        // Find the label with most votes
        // 找出得票最多的标签
        int predicted_label = std::distance(labelCount.begin(),
                                        std::max_element(labelCount.begin(), labelCount.end()));
        int true_label = test_samples[t].label;

        // Update accuracy statistics
        // 更新准确率统计
        if (predicted_label == true_label) {
            correct_predictions++;
        }

        // Show progress every 100 samples
        // 每处理100个样本显示一次进度
        if (t % 100 == 0) {
            float current_accuracy = (float)correct_predictions / (t + 1) * 100.0f;
            std::cout << "\rProcessing: " << t << "/" << num_testsamples
                     << " (Current Accuracy: " << current_accuracy << "%)" << std::flush;
        }
    }

    // Calculate final results
    // 计算最终结果
    float accuracy = (float)correct_predictions / num_testsamples * 100.0f;
    auto end_time = std::chrono::high_resolution_clock::now();
    auto duration = std::chrono::duration_cast<std::chrono::milliseconds>(end_time - start_time);

    // Print final results
    // 打印最终结果
    std::cout << "\n\nFinal Results:" << std::endl;
    std::cout << "Total test samples: " << num_testsamples << std::endl;
    std::cout << "Correct predictions: " << correct_predictions << std::endl;
    std::cout << "Accuracy: " << accuracy << "%" << std::endl;
    std::cout << "Total execution time: " << duration.count() / 1000.0 << " seconds" << std::endl;

    // Cleanup allocated memory
    // 清理分配的内存
    CUDA_CHECK(cudaFree(d_test_image));
    CUDA_CHECK(cudaFree(d_distances));
    CUDA_CHECK(cudaFree(d_sort_labels));
    CUDA_CHECK(cudaFreeHost(h_distances));
    CUDA_CHECK(cudaFreeHost(h_labels));
    CUDA_CHECK(cudaFreeHost(h_train_images));
    CUDA_CHECK(cudaFreeHost(h_train_labels));
    CUDA_CHECK(cudaFree(d_train_images));
    CUDA_CHECK(cudaFree(d_train_labels));

    return 0;
}

In [ ]:
%cuda_group_run --group "knn" --compiler-args "-O3 -g -std=c++20 -arch=sm_75"